In [118]:
from tsdb_client import TSDBClient
import pandas as pd
import trading_calendars as tc
import pytz
xtai = tc.get_calendar("XTAI")
import datetime


In [128]:
cli = TSDBClient(
    host="128.110.25.99",
    port=5432,
    user="chiubj",
    password="bunnygood",
    db="accountdb"
)

In [27]:
strategy_df = cli.execute_query('''
SELECT * FROM dealer.strategy
''', 
    out_type='df')

In [87]:
trade_df = cli.execute_query('''
    select 
        t_st.name as 策略, 
        t_pos.code as 代碼,
        t_pos.position as 部位,
        t_pos.avg_price as 成本價,
        t_quote.close as 收盤價,
        (position * (close - avg_price))*1000 as 損益, 
        (close / avg_price - 1) * 100 as "損益(%)",
        t_pos.trade_date

    from (
        select 
            strategy, 
            code,
            trade_date,
            sum((case when action = 'B' then 1 else -1 end) * qty) as position,
            sum(price*qty) / sum(qty) as avg_price
        from dealer.trades
        group by strategy, code, trade_date
    )t_pos
left join public.quote_snapshots t_quote on t_pos.code = t_quote.code
left join dealer.strategy t_st on t_pos.strategy = t_st.id 

order by t_pos.strategy, t_pos.code
''', 
    out_type='df')

In [88]:
status_list = []

for i in range(len(trade_df)):
    if date(datetime.now().year, datetime.now().month, datetime.now().day) == date(trade_df.iloc[i]['trade_date'].year, trade_df.iloc[i]['trade_date'].month, trade_df.iloc[i]['trade_date'].day):
        strategy_id = int(strategy_df[strategy_df['name'] == trade_df.iloc[i]['策略']]['id'])
        holding_period = int(strategy_df[strategy_df['name'] == trade_df.iloc[i]['策略']]['holding_period'])
        in_date = trade_df.iloc[i]['trade_date']
        out_date = datetime.date(xtai.sessions_window(pd.Timestamp(in_date.strftime("%Y-%m-%d")), holding_period)[-1])
        
        status_list.append([strategy_id, 'S', trade_df.iloc[i]['代碼'], trade_df.iloc[i]['成本價'], trade_df.iloc[i]['部位'], in_date, out_date])


In [90]:
cli.execute_values_df(df = pd.DataFrame(status_list, columns=['strategy', 'security_type', 'code', 'cost', 'qty', 'in_date', 'out_date']), table = 'dealer.status')

In [111]:

cli.execute_query(f'''
SELECT * FROM dealer.status where out_date = '{datetime.now().strftime("%Y-%m-%d")}'
''', out_type='df')


,strategy,security_type,code,cost,qty,in_date,out_date


In [130]:

cur_status = cli.execute_query(f'''
SELECT * FROM dealer.status where out_date = '2023-05-23'
''', out_type='df')


In [123]:
cur_status

,strategy,security_type,code,cost,qty,in_date,out_date
0,1,S,1504,48.7,13,2023-05-09,2023-05-23
1,2,S,3479,119.0,2,2023-05-09,2023-05-23


In [144]:
cur_contracts_df = cli.execute_query(f'''
SELECT * FROM sino.contracts where code = '{row.code}'
''', out_type='df')

In [155]:

counter = 1
sell_signal = []

for index, row in cur_status.iterrows():
    cur_contracts_df = cli.execute_query(f'''
        SELECT * FROM sino.contracts where code = '{row.code}'
        ''', out_type='df')
    if row.security_type == 'S': security_type = 'Stock' 
    if row.security_type == 'F': security_type = 'Futures'
    action = 'S' if row.qty > 0 else 'B'
    sell_signal.append([f'N{counter}', security_type, str(datetime.datetime.now().timestamp()), row.code, 'ROD', action, abs(row.qty), '%.2f'%cur_contracts_df['limit_down'].values[0]])
    counter += 1



In [156]:
sell_signal

[['N1', 'Stock', '1684131448.364166', '1504', 'ROD', 'S', 13, '40.95'],
 ['N2', 'Stock', '1684131448.508667', '3479', 'ROD', 'S', 2, '112.00']]